In [ ]:
# This cell imports all required libraries and setups some helpers
import micropip
await micropip.install(["ipywidgets"])

from IPython.display import display, JSON
from ipywidgets import widgets

import js
import json
import os


class Camunda:
    @staticmethod
    def get(path):
        request = js.XMLHttpRequest.new()
        request.open("GET", f"{os.environ['CAMUNDA_ENGINE_API']}{path}", False)
        request.send(None)
        assert request.status in [200, 204], request.responseText
        return json.loads(request.responseText or 'null')
    
    @staticmethod
    def delete(path):
        request = js.XMLHttpRequest.new()
        request.open("DELETE", f"{os.environ['CAMUNDA_ENGINE_API']}{path}", False)
        request.setRequestHeader("X-XSRF-TOKEN", os.environ['CAMUNDA_CSRF_TOKEN'])
        request.send(None)
        assert request.status in [200, 204], request.responseText
        return json.loads(request.responseText or 'null')

In [ ]:
button = widgets.Button(description='Reset deployments')
out = widgets.Output()
def on_button_clicked(_):
    data = Camunda.get("/deployment/")
    for deployment in data:
        path = f"/deployment/{deployment['id']}?cascade=true&skipCustomListeners=true&skipIoMappings=true"
        with out:
            display(path)
        Camunda.delete(path)
button.on_click(on_button_clicked)
widgets.VBox([button,out])